**1. 📥 Загрузка JSON-строки → Python-объект (dict)**

In [60]:
import json

json_str = '{"name": "Иван", "age": 30}' # str
data = json.loads(json_str)  # str → dict
print(data["name"])  # → Иван

Иван


**2. 📤 Преобразование Python-объекта → JSON-строка**

`ensure_ascii=False` — сохраняет кириллицу

`indent=2` — форматированный вывод, отступ слева

In [61]:
import json

data = {"name": "Иван", "age": 30} # dict
json_str = json.dumps(data, ensure_ascii=False, indent=0) # dict → str
print(json_str)

{
"name": "Иван",
"age": 30
}


**3. 🛰 Использование с API (например, HH.ru)**

In [62]:
import requests
import json

url = "https://api.hh.ru/employers"
params = {"text": "ДИКСИ, группа компаний", "only_with_vacancies": True}
response = requests.get(url, params=params) # <class 'requests.models.Response'>

if response.ok:
    data = response.json() # JSON → dict
    print(type(data))
    print(type(json.dumps(data, ensure_ascii=False, indent=2))) # dict → str
    print(json.dumps(data, ensure_ascii=False, indent=2))


<class 'dict'>
<class 'str'>
{
  "items": [
    {
      "id": "2746",
      "name": "ДИКСИ, группа компаний",
      "url": "https://api.hh.ru/employers/2746",
      "alternate_url": "https://hh.ru/employer/2746",
      "logo_urls": {
        "original": "https://img.hhcdn.ru/employer-logo-original/1393191.png",
        "240": "https://img.hhcdn.ru/employer-logo/7192548.png",
        "90": "https://img.hhcdn.ru/employer-logo/7192547.png"
      },
      "vacancies_url": "https://api.hh.ru/vacancies?employer_id=2746",
      "open_vacancies": 3616
    }
  ],
  "found": 1,
  "pages": 1,
  "page": 0,
  "per_page": 20
}


**4. 🔍 Обработка вложенного JSON (вложенные dict и list)**

In [63]:
# Допустим, data = response.json() и содержит список компаний:
for item in data["items"]:
    print("Компания:", item["name"])
    print("Вакансий:", item.get("open_vacancies"))

    # Проверка наличия логотипа
    if item.get("logo_urls"):
        print("Логотип 90:", item["logo_urls"].get("90"))
    else:
        print("Логотип отсутствует")

    print("---")

Компания: ДИКСИ, группа компаний
Вакансий: 3616
Логотип 90: https://img.hhcdn.ru/employer-logo/7192547.png
---


**5. 💾 Загрузка из JSON-файла**

In [64]:
with open("/content/diksi_employers.json", "r", encoding="utf-8") as f:
    data = json.load(f)

print(data)

[{'id': '2746', 'name': 'ДИКСИ, группа компаний', 'open_vacancies': 3612, 'logo_urls': {'original': 'https://img.hhcdn.ru/employer-logo-original/1393191.png', '240': 'https://img.hhcdn.ru/employer-logo/7192548.png', '90': 'https://img.hhcdn.ru/employer-logo/7192547.png'}}]


**6. 💽 Сохранение в JSON-файл**

In [55]:
with open("file.json", "w", encoding="utf-8") as f:
    json.dump(data, f, ensure_ascii=False, indent=2)

**7. 📑 Частые методы при работе с JSON-объектами (dict)**

In [ ]:
data.get("ключ")          # безопасно получить значение (вернёт None, если ключа нет)
data["ключ"]              # прямой доступ (ошибка, если ключа нет)
data.items()              # пары ключ–значение
data.keys()               # все ключи
data.values()             # все значения

**8. 🧪 Пример полной цепочки: API → Python → Фильтрация → JSON-файл**

In [65]:
import requests, json

url = "https://api.hh.ru/employers"
params = {"text": "ДИКСИ, группа компаний", "only_with_vacancies": True}
response = requests.get(url, params=params)

if response.ok:
    data = response.json()

    filtered = [
        {
            "id": emp["id"],
            "name": emp["name"],
            "open_vacancies": emp.get("open_vacancies"),
            "logo_urls": emp.get("logo_urls", {}),
        }
        for emp in data["items"]
    ]

    with open("diksi_employers.json", "w", encoding="utf-8") as f:
        json.dump(filtered, f, ensure_ascii=False, indent=2)

    print("Сохранено:", len(filtered), "работодателей")


Сохранено: 1 работодателей


**9. 🔍 Что такое Response?**

Это объект, который представляет ответ от сервера. С его помощью можно получить:

```
Атрибут / Метод                       Что делает
response.status_code	              Код ответа HTTP (200, 404, 500 и т.д.)
response.ok	                       True, если статус 200–299
response.text	                     Ответ как текст
response.content	                  Ответ как байты (например, картинка или PDF)
response.json()	                   Преобразует тело ответа в dict (если это JSON)
response.headers	                  Заголовки ответа
response.url	                      URL, по которому был сделан запрос
response.raise_for_status()	       Бросает исключение, если статус ответа ≥ 400
```

In [66]:
import requests

response = requests.get("https://api.hh.ru/employers", params={"text": "Газпромбанк"})

print(type(response))  # <class 'requests.models.Response'>
print(response.status_code)  # 200
print(response.ok)  # True
print(response.url)  # финальный URL
print(response.headers)  # словарь заголовков


<class 'requests.models.Response'>
200
True
https://api.hh.ru/employers?text=%D0%93%D0%B0%D0%B7%D0%BF%D1%80%D0%BE%D0%BC%D0%B1%D0%B0%D0%BD%D0%BA
{'Server': 'ddos-guard', 'Connection': 'keep-alive', 'Keep-Alive': 'timeout=60', 'Set-Cookie': '__ddg8_=gCBMCoLrkwJhLsbk; Domain=.hh.ru; Path=/; Expires=Wed, 30-Apr-2025 10:41:43 GMT, __ddg10_=1746008503; Domain=.hh.ru; Path=/; Expires=Wed, 30-Apr-2025 10:41:43 GMT, __ddg9_=34.28.220.56; Domain=.hh.ru; Path=/; Expires=Wed, 30-Apr-2025 10:41:43 GMT, __ddg1_=fMWchuKezLFLjMyyWxSO; Domain=.hh.ru; HttpOnly; Path=/; Expires=Thu, 30-Apr-2026 10:21:43 GMT', 'Date': 'Wed, 30 Apr 2025 10:21:43 GMT', 'Content-Type': 'application/json; charset=UTF-8', 'Vary': 'Accept-Encoding', 'X-Request-Id': '1746008503803b7cb4fadd52557294d6, 1746008503803b7cb4fadd52557294d6', 'Cache-Control': 'max-age=0, private, must-revalidate', 'Expires': 'Wed, 19 Mar 2025 10:21:43 GMT', 'Access-Control-Allow-Origin': '*', 'Access-Control-Expose-Headers': 'Location, ETag, Date, Expir